<a href="https://colab.research.google.com/github/Tonmf/ProyectoIML/blob/master/Casos/Caso_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reporte de MLT de Barcelona Telco

## Caso 5: Fábrica Milwaukee (aka Bosch)


## Data

Source: https://www.kaggle.com/c/bosch-production-line-performance

In [ ]:
import os              # Package to use directory command to list files
import numpy as np     # linear algebra
import pandas as pd    # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns  # Used for plotting the graph
import matplotlib.pyplot as plt
import re

from scipy import stats # for Statistics
from statistics import mean
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from matplotlib import rcParams
%matplotlib inline

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

In [5]:
data = pd.read_csv('/content/gdrive/My Drive/IML/Proyecto Final/train_numeric.csv',nrows=4000)

## Data Exploration

In [6]:
print(data.shape)
data.head(5)

(4000, 970)


,Id,L0_S0_F0,L0_S0_F2,L0_S0_F4,L0_S0_F6,L0_S0_F8,L0_S0_F10,L0_S0_F12,L0_S0_F14,L0_S0_F16,L0_S0_F18,L0_S0_F20,L0_S0_F22,L0_S1_F24,L0_S1_F28,L0_S2_F32,L0_S2_F36,L0_S2_F40,L0_S2_F44,L0_S2_F48,L0_S2_F52,L0_S2_F56,L0_S2_F60,L0_S2_F64,L0_S3_F68,L0_S3_F72,L0_S3_F76,L0_S3_F80,L0_S3_F84,L0_S3_F88,L0_S3_F92,L0_S3_F96,L0_S3_F100,L0_S4_F104,L0_S4_F109,L0_S5_F114,L0_S5_F116,L0_S6_F118,L0_S6_F122,L0_S6_F132,...,L3_S45_F4126,L3_S45_F4128,L3_S45_F4130,L3_S45_F4132,L3_S47_F4138,L3_S47_F4143,L3_S47_F4148,L3_S47_F4153,L3_S47_F4158,L3_S47_F4163,L3_S47_F4168,L3_S47_F4173,L3_S47_F4178,L3_S47_F4183,L3_S47_F4188,L3_S48_F4193,L3_S48_F4196,L3_S48_F4198,L3_S48_F4200,L3_S48_F4202,L3_S48_F4204,L3_S49_F4206,L3_S49_F4211,L3_S49_F4216,L3_S49_F4221,L3_S49_F4226,L3_S49_F4231,L3_S49_F4236,L3_S50_F4241,L3_S50_F4243,L3_S50_F4245,L3_S50_F4247,L3_S50_F4249,L3_S50_F4251,L3_S50_F4253,L3_S51_F4256,L3_S51_F4258,L3_S51_F4260,L3_S51_F4262,Response
0,4,0.030,-0.034,-0.197,-0.179,0.118,0.116,-0.015,-0.032,0.020,0.083,-0.273,-0.273,-0.271,0.167,-0.213,-0.023,-0.192,-0.088,0.001,0.0,0.01,-0.223,-0.030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.001,-0.004,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,7,0.088,0.086,0.003,-0.052,0.161,0.025,-0.015,-0.072,-0.225,-0.147,0.250,0.250,0.057,-0.079,-0.013,0.011,0.008,-0.060,-0.005,0.0,0.01,-0.223,-0.077,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.073,0.138,-0.336,0.506,-0.13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,9,-0.036,-0.064,0.294,0.330,0.074,0.161,0.022,0.128,-0.026,-0.046,-0.253,-0.253,0.147,-0.007,-0.013,0.120,0.008,-0.231,0.005,0.0,0.01,0.050,0.056,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.038,-0.001,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,11,-0.055,-0.086,0.294,0.330,0.118,0.025,0.030,0.168,-0.169,-0.099,0.042,0.042,-0.012,-0.046,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.009,-0.272,-0.051,0.037,0.004,0.0,-0.081,0.311,0.003,0.021,0.015,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [ ]:
data.describe()

,Id,L0_S0_F0,L0_S0_F2,L0_S0_F4,L0_S0_F6,L0_S0_F8,L0_S0_F10,L0_S0_F12,L0_S0_F14,L0_S0_F16,L0_S0_F18,L0_S0_F20,L0_S0_F22,L0_S1_F24,L0_S1_F28,L0_S2_F32,L0_S2_F36,L0_S2_F40,L0_S2_F44,L0_S2_F48,L0_S2_F52,L0_S2_F56,L0_S2_F60,L0_S2_F64,L0_S3_F68,L0_S3_F72,L0_S3_F76,L0_S3_F80,L0_S3_F84,L0_S3_F88,L0_S3_F92,L0_S3_F96,L0_S3_F100,L0_S4_F104,L0_S4_F109,L0_S5_F114,L0_S5_F116,L0_S6_F118,L0_S6_F122,L0_S6_F132,...,L3_S45_F4126,L3_S45_F4128,L3_S45_F4130,L3_S45_F4132,L3_S47_F4138,L3_S47_F4143,L3_S47_F4148,L3_S47_F4153,L3_S47_F4158,L3_S47_F4163,L3_S47_F4168,L3_S47_F4173,L3_S47_F4178,L3_S47_F4183,L3_S47_F4188,L3_S48_F4193,L3_S48_F4196,L3_S48_F4198,L3_S48_F4200,L3_S48_F4202,L3_S48_F4204,L3_S49_F4206,L3_S49_F4211,L3_S49_F4216,L3_S49_F4221,L3_S49_F4226,L3_S49_F4231,L3_S49_F4236,L3_S50_F4241,L3_S50_F4243,L3_S50_F4245,L3_S50_F4247,L3_S50_F4249,L3_S50_F4251,L3_S50_F4253,L3_S51_F4256,L3_S51_F4258,L3_S51_F4260,L3_S51_F4262,Response
count,4000.000000,2325.000000,2325.000000,2325.000000,2325.000000,2325.000000,2325.000000,2325.000000,2325.000000,2325.000000,2325.000000,2325.000000,2325.000000,2327.000000,2327.000000,1166.000000,1166.000000,1166.000000,1166.000000,1166.000000,1166.0,1166.000000,1166.000000,1166.000000,1160.000000,1160.000000,1160.000000,1160.000000,1160.000000,1160.0,1160.000000,1160.000000,1160.000000,1139.000000,1139.000000,1189.000000,1189.000000,1160.000000,1160.000000,1160.000000,...,194.0,194.0,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.0,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.0,194.000000,194.000000,92.000000,92.000000,92.000000,92.000000,92.0,92.0,92.000000,102.000000,102.000000,102.000000,102.000000,102.0,102.000000,102.000000,194.000000,194.000000,194.000000,194.000000,4000.00000
mean,4018.952500,-0.001055,-0.001595,0.004938,0.004402,0.001665,0.007874,0.000389,0.002804,-0.001648,-0.002217,0.001983,0.001995,0.002849,0.002853,-0.004338,0.015549,-0.004350,-0.000332,-0.000110,0.0,-0.001664,0.000624,0.006369,0.000466,0.001101,0.002017,0.002544,0.000182,0.0,0.001672,-0.008123,-0.001122,0.001487,0.000307,-0.000683,-0.007216,-0.016172,0.021811,-0.000716,...,0.0,0.0,0.000010,0.000077,0.005753,-0.000124,0.006572,-0.001381,0.001041,0.019814,0.006572,0.0,0.000222,0.000082,0.000072,-0.023242,0.008294,-0.000134,0.0,0.000149,0.000093,-0.001011,0.001978,-0.000011,0.000304,0.0,0.0,0.022739,-0.025118,-0.004627,-0.000020,0.000010,0.0,0.000186,-0.009804,-0.000005,0.000005,0.000155,0.000196,0.00575
std,2307.764835,0.078488,0.091779,0.213230,0.212953,0.094116,0.160903,0.019176,0.103750,0.113865,0.110104,0.201989,0.201972,0.103925,0.108682,0.050767,0.209693,0.068348,0.264412,0.006543,0.0,0.056821,0.262852,0.061439,0.046309,0.250478,0.054078,0.271901,0.006336,0.0,0.052887,0.242717,0.053020,0.054708,0.014361,0.127981,0.150973,0.248974,0.345922,0.106862,...,0.0,0.0,0.000101,0.000938,0.096644,0.003160,0.103856,0.084464,0.176276,0.102905,0.103856,0.0,0.001904,0.000570,0.001005,0.404670,0.131445,0.061192,0.0,0.001040,0.001044,0.000104,0.051298,0.000104,0.002237,0.0,0.0,0.250418,0.118425,0.150980,0.000139,0.000099,0.0,0.001031,0.251041,0.000072,0.000072,0.001549,0.001686,0.07562
min,4.000000,-0.291000,-0.362000,-0.397000,-0.397000,-0.404000,-0.566000,-0.044000,-0.232000,-0.393000,-0.334000,-0.340000,-0.340000,-0.287000,-0.253000,-0.213000,-0.208000,-0.192000,-0.273000,-0.037000,0.0,-0.190000,-0.266000,-0.084000,-0.191000,-0.274000,-0.051000,-0.350000,-0.036000,0.0,-0.181000,-0.290000,-0.088000,-0.142000,-0.046000,-0.342000,-0.207000,-0.336000,-0.323000,-0.142000,...,0.0,0.0,0.000000,0.000000,-0.217000,-0.010000,-0.224000,-0.239000,-0.418000,-0.287000,-0.224000,0.0,0.000000,0.000000,0.000000,-0.520000,-0.327000,-0.573000,0.0,0.000000,0.000000,-0.002000,-0.075000,-0.001000,0.000000,0.0,0.0,-0.249000,-0.531000,-0.257000,-0.001000,0.000000,0.0,0.000000,-0.250000,-0.001000,0.000000,0.000000,0.000000,0.00000
25%,2058.000000,-0.055000,-0.056000,-0.179000,-0.179000,-0.056000,-0.0

In [ ]:
data.columns

Index(['Id', 'L0_S0_F0', 'L0_S0_F2', 'L0_S0_F4', 'L0_S0_F6', 'L0_S0_F8',
       'L0_S0_F10', 'L0_S0_F12', 'L0_S0_F14', 'L0_S0_F16',
       ...
       'L3_S50_F4245', 'L3_S50_F4247', 'L3_S50_F4249', 'L3_S50_F4251',
       'L3_S50_F4253', 'L3_S51_F4256', 'L3_S51_F4258', 'L3_S51_F4260',
       'L3_S51_F4262', 'Response'],
      dtype='object', length=970)

In [ ]:
data.dtypes

Id                int64
L0_S0_F0        float64
L0_S0_F2        float64
L0_S0_F4        float64
L0_S0_F6        float64
                 ...   
L3_S51_F4256    float64
L3_S51_F4258    float64
L3_S51_F4260    float64
L3_S51_F4262    float64
Response          int64
Length: 970, dtype: object

In [7]:
totalt = data.isnull().sum().sort_values(ascending=False)
percent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([totalt, percent], axis=1, keys=['Total', 'Percent'])
missing_data

,Total,Percent
L1_S25_F2181,3997,0.99925
L1_S25_F2184,3997,0.99925
L1_S25_F2187,3997,0.99925
L1_S25_F2190,3997,0.99925
L1_S25_F2193,3997,0.99925
...,...,...
L3_S37_F3946,204,0.05100
L3_S37_F3948,204,0.05100
L3_S37_F3950,204,0.05100
Response,0,0.00000


In [8]:
missing2 = missing_data.loc[(missing_data['Percent']>0.6)]
missing2

,Total,Percent
L1_S25_F2181,3997,0.99925
L1_S25_F2184,3997,0.99925
L1_S25_F2187,3997,0.99925
L1_S25_F2190,3997,0.99925
L1_S25_F2193,3997,0.99925
...,...,...
L0_S7_F138,2832,0.70800
L0_S7_F142,2832,0.70800
L0_S7_F136,2832,0.70800
L0_S5_F114,2811,0.70275


In [ ]:
796/970 *100

82.06185567010309

In [9]:
missing2.index

Index(['L1_S25_F2181', 'L1_S25_F2184', 'L1_S25_F2187', 'L1_S25_F2190',
       'L1_S25_F2193', 'L1_S25_F2196', 'L1_S25_F2199', 'L1_S25_F2202',
       'L1_S25_F2682', 'L1_S25_F2707',
       ...
       'L0_S2_F44', 'L0_S2_F52', 'L0_S2_F56', 'L0_S2_F60', 'L0_S2_F64',
       'L0_S7_F138', 'L0_S7_F142', 'L0_S7_F136', 'L0_S5_F114', 'L0_S5_F116'],
      dtype='object', length=796)

In [ ]:
df_agrup = data.groupby(by=['Response'])
df_agrup.count()

,Id,L0_S0_F0,L0_S0_F2,L0_S0_F4,L0_S0_F6,L0_S0_F8,L0_S0_F10,L0_S0_F12,L0_S0_F14,L0_S0_F16,L0_S0_F18,L0_S0_F20,L0_S0_F22,L0_S1_F24,L0_S1_F28,L0_S2_F32,L0_S2_F36,L0_S2_F40,L0_S2_F44,L0_S2_F48,L0_S2_F52,L0_S2_F56,L0_S2_F60,L0_S2_F64,L0_S3_F68,L0_S3_F72,L0_S3_F76,L0_S3_F80,L0_S3_F84,L0_S3_F88,L0_S3_F92,L0_S3_F96,L0_S3_F100,L0_S4_F104,L0_S4_F109,L0_S5_F114,L0_S5_F116,L0_S6_F118,L0_S6_F122,L0_S6_F132,...,L3_S45_F4124,L3_S45_F4126,L3_S45_F4128,L3_S45_F4130,L3_S45_F4132,L3_S47_F4138,L3_S47_F4143,L3_S47_F4148,L3_S47_F4153,L3_S47_F4158,L3_S47_F4163,L3_S47_F4168,L3_S47_F4173,L3_S47_F4178,L3_S47_F4183,L3_S47_F4188,L3_S48_F4193,L3_S48_F4196,L3_S48_F4198,L3_S48_F4200,L3_S48_F4202,L3_S48_F4204,L3_S49_F4206,L3_S49_F4211,L3_S49_F4216,L3_S49_F4221,L3_S49_F4226,L3_S49_F4231,L3_S49_F4236,L3_S50_F4241,L3_S50_F4243,L3_S50_F4245,L3_S50_F4247,L3_S50_F4249,L3_S50_F4251,L3_S50_F4253,L3_S51_F4256,L3_S51_F4258,L3_S51_F4260,L3_S51_F4262
Response,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,3977,2316,2316,2316,2316,2316,2316,2316,2316,2316,2316,2316,2316,2318,2318,1164,1164,1164,1164,1164,1164,1164,1164,1164,1153,1153,1153,1153,1153,1153,1153,1153,1153,1133,1133,1186,1186,1154,1154,1154,...,193,193,193,193,193,193,193,193,193,193,193,193,193,193,193,193,193,193,193,193,193,193,91,91,91,91,91,91,91,102,102,102,102,102,102,102,193,193,193,193
1,23,9,9,9,9,9,9,9,9,9,9,9,9,9,9,2,2,2,2,2,2,2,2,2,7,7,7,7,7,7,7,7,7,6,6,3,3,6,6,6,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,1,1,1,1


In [ ]:
23/4000 *100

0.575

De este recuento podemos sacar un par de cosas interesantes: 

1.   Sólo hay 23 casos en que tenemos una pieza mala
2.   De un vistazo rápido vemos que en cierta líneas hay más 1's que en otras.

In [27]:
df3 = df2.drop(labels=missing2.index,axis=1)

In [28]:
df3.head()

,Id,L0_S0_F0,L0_S0_F2,L0_S0_F4,L0_S0_F6,L0_S0_F8,L0_S0_F10,L0_S0_F12,L0_S0_F14,L0_S0_F16,L0_S0_F18,L0_S0_F20,L0_S0_F22,L0_S1_F24,L0_S1_F28,L0_S8_F144,L0_S8_F146,L0_S8_F149,L3_S29_F3315,L3_S29_F3318,L3_S29_F3321,L3_S29_F3324,L3_S29_F3327,L3_S29_F3330,L3_S29_F3333,L3_S29_F3336,L3_S29_F3339,L3_S29_F3342,L3_S29_F3345,L3_S29_F3348,L3_S29_F3351,L3_S29_F3354,L3_S29_F3357,L3_S29_F3360,L3_S29_F3367,L3_S29_F3370,L3_S29_F3373,L3_S29_F3376,L3_S29_F3379,L3_S29_F3382,...,L3_S30_F3809,L3_S30_F3814,L3_S30_F3819,L3_S30_F3824,L3_S30_F3829,L3_S33_F3855,L3_S33_F3857,L3_S33_F3859,L3_S33_F3861,L3_S33_F3863,L3_S33_F3865,L3_S33_F3867,L3_S33_F3869,L3_S33_F3871,L3_S33_F3873,L3_S34_F3876,L3_S34_F3878,L3_S34_F3880,L3_S34_F3882,L3_S35_F3884,L3_S35_F3889,L3_S35_F3894,L3_S35_F3896,L3_S35_F3898,L3_S35_F3903,L3_S35_F3908,L3_S35_F3913,L3_S36_F3918,L3_S36_F3920,L3_S36_F3922,L3_S36_F3924,L3_S36_F3926,L3_S36_F3930,L3_S36_F3934,L3_S36_F3938,L3_S37_F3944,L3_S37_F3946,L3_S37_F3948,L3_S37_F3950,Response
0,4,0.030,-0.034,-0.197,-0.179,0.118,0.116,-0.015,-0.032,0.020,0.083,-0.273,-0.273,-0.271,0.167,-0.157,0.0,0.001,0.030,-0.047,-0.119,0.040,-0.067,-0.058,0.044,0.044,0.010,0.041,-0.072,-0.024,-0.026,-0.257,0.250,0.218,0.040,0.044,0.090,0.036,-0.117,0.054,...,-0.060,0.013,-0.027,0.013,0.005,-0.119,-0.102,-0.048,-0.119,-0.119,0.026,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.009,-0.082,-0.022,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0
1,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.084,-0.030,-0.127,-0.174,0.145,0.152,-0.006,-0.059,-0.064,-0.031,0.023,-0.010,0.113,0.238,-0.050,0.218,-0.174,-0.059,0.085,-0.015,-0.244,-0.050,...,-0.041,0.013,-0.019,0.013,-0.049,-0.029,-0.026,-0.210,-0.029,-0.029,-0.047,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.011,0.030,-0.028,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0
2,7,0.088,0.086,0.003,-0.052,0.161,0.025,-0.015,-0.072,-0.225,-0.147,0.250,0.250,0.057,-0.079,-0.157,0.0,0.001,0.049,-0.030,-0.099,0.070,0.014,0.004,0.006,0.064,0.040,-0.005,-0.129,-0.138,0.101,0.238,-0.017,-0.115,0.070,0.064,-0.223,0.023,0.223,0.047,...,0.115,0.013,0.051,0.013,-0.069,0.199,0.191,0.236,0.199,0.199,0.108,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.036,0.030,-0.091,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0
3,9,-0.036,-0.064,0.294,0.330,0.074,0.161,0.022,0.128,-0.026,-0.046,-0.253,-0.253,0.147,-0.007,0.343,0.0,0.001,-0.084,-0.139,-0.039,-0.207,0.024,0.025,0.038,-0.033,0.014,0.020,-0.066,-0.038,0.078,-0.099,-0.017,0.218,-0.207,-0.033,0.137,0.030,-0.263,-0.024,...,0.057,0.013,0.025,0.013,-0.014,0.017,0.031,0.165,0.017,0.017,0.035,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.192,-0.030,-0.192,-0.030,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,11,-0.055,-0.086,0.294,0.330,0.118,0.025,0.030,0.168,-0.169,-0.099,0.042,0.042,-0.012,-0.046,-0.157,0.0,0.001,-0.147,-0.082,-0.091,-0.137,-0.049,-0.063,-0.163,-0.111,0.053,0.033,0.127,0.089,-0.167,-0.257,-0.050,0.218,-0.137,-0.111,-0.347,-0.149,0.099,-0.128,...,0.004,0.013,0.001,0.013,-0.056,-0.074,-0.049,0.147,-0.074,-0.074,-0.006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.092,0.083,-0.092,0.083,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [29]:
df_agrup3 = df3.groupby(by=['Response'])
df_agrup3.count()

,Id,L0_S0_F0,L0_S0_F2,L0_S0_F4,L0_S0_F6,L0_S0_F8,L0_S0_F10,L0_S0_F12,L0_S0_F14,L0_S0_F16,L0_S0_F18,L0_S0_F20,L0_S0_F22,L0_S1_F24,L0_S1_F28,L0_S8_F144,L0_S8_F146,L0_S8_F149,L3_S29_F3315,L3_S29_F3318,L3_S29_F3321,L3_S29_F3324,L3_S29_F3327,L3_S29_F3330,L3_S29_F3333,L3_S29_F3336,L3_S29_F3339,L3_S29_F3342,L3_S29_F3345,L3_S29_F3348,L3_S29_F3351,L3_S29_F3354,L3_S29_F3357,L3_S29_F3360,L3_S29_F3367,L3_S29_F3370,L3_S29_F3373,L3_S29_F3376,L3_S29_F3379,L3_S29_F3382,...,L3_S30_F3804,L3_S30_F3809,L3_S30_F3814,L3_S30_F3819,L3_S30_F3824,L3_S30_F3829,L3_S33_F3855,L3_S33_F3857,L3_S33_F3859,L3_S33_F3861,L3_S33_F3863,L3_S33_F3865,L3_S33_F3867,L3_S33_F3869,L3_S33_F3871,L3_S33_F3873,L3_S34_F3876,L3_S34_F3878,L3_S34_F3880,L3_S34_F3882,L3_S35_F3884,L3_S35_F3889,L3_S35_F3894,L3_S35_F3896,L3_S35_F3898,L3_S35_F3903,L3_S35_F3908,L3_S35_F3913,L3_S36_F3918,L3_S36_F3920,L3_S36_F3922,L3_S36_F3924,L3_S36_F3926,L3_S36_F3930,L3_S36_F3934,L3_S36_F3938,L3_S37_F3944,L3_S37_F3946,L3_S37_F3948,L3_S37_F3950
Response,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,3977,2316,2316,2316,2316,2316,2316,2316,2316,2316,2316,2316,2316,2318,2318,2315,2315,2315,3767,3767,3767,3767,3767,3767,3767,3767,3767,3767,3767,3767,3767,3767,3767,3767,3767,3767,3767,3767,3767,3767,...,3771,3771,3771,3771,3771,3771,3757,3757,3757,3757,3757,3757,3757,3757,3757,3757,3756,3756,3756,3756,1897,1897,1897,1897,1897,1897,1897,1897,1878,1878,1878,1878,1878,1878,1878,1878,3775,3775,3775,3775
1,23,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,...,21,21,21,21,21,21,17,17,17,17,17,17,17,17,17,17,17,17,17,17,12,12,12,12,12,12,12,12,9,9,9,9,9,9,9,9,21,21,21,21


In [30]:
totalt = df3.isnull().sum().sort_values(ascending=False)
percent = (df3.isnull().sum()/df3.isnull().count()).sort_values(ascending=False)
missing_data3 = pd.concat([totalt, percent], axis=1, keys=['Total', 'Percent'])
missing_data3

,Total,Percent
L3_S36_F3918,2113,0.52825
L3_S36_F3938,2113,0.52825
L3_S36_F3934,2113,0.52825
L3_S36_F3930,2113,0.52825
L3_S36_F3926,2113,0.52825
...,...,...
L3_S37_F3944,204,0.05100
L3_S37_F3946,204,0.05100
L3_S37_F3948,204,0.05100
Response,0,0.00000


In [45]:
df4 = df3.fillna(df3.mean())

In [46]:
totalt = df4.isnull().sum().sort_values(ascending=False)
percent = (df4.isnull().sum()/df4.isnull().count()).sort_values(ascending=False)
missing_data4 = pd.concat([totalt, percent], axis=1, keys=['Total', 'Percent'])
missing_data4

,Total,Percent
Response,0,0.0
L3_S29_F3439,0,0.0
L3_S29_F3467,0,0.0
L3_S29_F3464,0,0.0
L3_S29_F3461,0,0.0
...,...,...
L3_S30_F3709,0,0.0
L3_S30_F3704,0,0.0
L3_S30_F3699,0,0.0
L3_S30_F3694,0,0.0


## Algoritmo de Machine Learning

In [50]:
missing_data4.index

Index(['Response', 'L3_S29_F3439', 'L3_S29_F3467', 'L3_S29_F3464',
       'L3_S29_F3461', 'L3_S29_F3458', 'L3_S29_F3455', 'L3_S29_F3452',
       'L3_S29_F3449', 'L3_S29_F3442',
       ...
       'L3_S30_F3734', 'L3_S30_F3729', 'L3_S30_F3724', 'L3_S30_F3719',
       'L3_S30_F3714', 'L3_S30_F3709', 'L3_S30_F3704', 'L3_S30_F3699',
       'L3_S30_F3694', 'Id'],
      dtype='object', length=174)

In [51]:
from sklearn.preprocessing import StandardScaler
target = df4['Response'].values

select_features = missing_data4.index


scaler = StandardScaler()
dfScaled = scaler.fit_transform(df4[select_features])

In [53]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(dfScaled, target, test_size=0.2, random_state=42)

In [54]:
from sklearn.ensemble import RandomForestClassifier  # importing model to use for our prediction
from sklearn.metrics import classification_report

RFC = RandomForestClassifier(max_depth = 5, min_samples_split = 4, n_estimators = 500,random_state = 20, n_jobs = -1)
RFC.fit(x_train, y_train) 
train_prediction = RFC.predict(x_train)
print("La accuracy es de {}.".format(accuracy_score(y_train,train_prediction)))
print(classification_report(y_train, train_prediction))

test_prediction = RFC.predict(x_test)
print("La accuracy es de {}.".format(accuracy_score(y_test, test_prediction)))
print(classification_report(y_test, test_prediction))

La accuracy es de 0.9959375.
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3179
           1       1.00      0.38      0.55        21

    accuracy                           1.00      3200
   macro avg       1.00      0.69      0.77      3200
weighted avg       1.00      1.00      1.00      3200

La accuracy es de 0.99875.
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       798
           1       1.00      0.50      0.67         2

    accuracy                           1.00       800
   macro avg       1.00      0.75      0.83       800
weighted avg       1.00      1.00      1.00       800



## Resultados

El resultado es sorprendentemente alto. Cabe recordar que sólo teníamos un 0.05% de casos en que teníamos pieza mala, por lo que accuracy menor de 99,95 no es del todo interesante ya que en la mayoría de casos si predice pieza buena la acierta... Probamos en equilibrar el número de piezas buenas y piezas malas del dataset.

Como sólo teníamos 23 casos de pieza mala, vamos a cargar más datos para poder generar un dataset suficientemente grande.

In [134]:
df5 = pd.read_csv('/content/gdrive/My Drive/IML/Proyecto Final/train_numeric.csv',nrows=100000)

In [135]:
df_agrup = df5.groupby(by=['Response'])
df_agrup.count()

,Id,L0_S0_F0,L0_S0_F2,L0_S0_F4,L0_S0_F6,L0_S0_F8,L0_S0_F10,L0_S0_F12,L0_S0_F14,L0_S0_F16,L0_S0_F18,L0_S0_F20,L0_S0_F22,L0_S1_F24,L0_S1_F28,L0_S2_F32,L0_S2_F36,L0_S2_F40,L0_S2_F44,L0_S2_F48,L0_S2_F52,L0_S2_F56,L0_S2_F60,L0_S2_F64,L0_S3_F68,L0_S3_F72,L0_S3_F76,L0_S3_F80,L0_S3_F84,L0_S3_F88,L0_S3_F92,L0_S3_F96,L0_S3_F100,L0_S4_F104,L0_S4_F109,L0_S5_F114,L0_S5_F116,L0_S6_F118,L0_S6_F122,L0_S6_F132,...,L3_S45_F4124,L3_S45_F4126,L3_S45_F4128,L3_S45_F4130,L3_S45_F4132,L3_S47_F4138,L3_S47_F4143,L3_S47_F4148,L3_S47_F4153,L3_S47_F4158,L3_S47_F4163,L3_S47_F4168,L3_S47_F4173,L3_S47_F4178,L3_S47_F4183,L3_S47_F4188,L3_S48_F4193,L3_S48_F4196,L3_S48_F4198,L3_S48_F4200,L3_S48_F4202,L3_S48_F4204,L3_S49_F4206,L3_S49_F4211,L3_S49_F4216,L3_S49_F4221,L3_S49_F4226,L3_S49_F4231,L3_S49_F4236,L3_S50_F4241,L3_S50_F4243,L3_S50_F4245,L3_S50_F4247,L3_S50_F4249,L3_S50_F4251,L3_S50_F4253,L3_S51_F4256,L3_S51_F4258,L3_S51_F4260,L3_S51_F4262
Response,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,99432,57625,57625,57625,57625,57625,57625,57625,57625,57625,57625,57625,57625,57641,57641,29010,29010,29010,29010,29010,29010,29010,29010,29010,28674,28674,28674,28674,28674,28674,28674,28674,28674,28730,28726,28979,28979,29072,29072,29072,...,5177,5177,5177,5177,5177,5175,5175,5175,5175,5175,5175,5175,5175,5175,5175,5175,5176,5176,5176,5176,5176,5176,2531,2531,2531,2531,2531,2531,2531,2650,2650,2650,2650,2650,2650,2650,5171,5171,5171,5171
1,568,290,290,290,290,290,290,290,290,290,290,290,290,289,289,151,151,151,151,151,151,151,151,151,139,139,139,139,139,139,139,139,139,150,150,141,141,143,143,143,...,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,24,24,24,24,24,24,11,11,12,12,12,12,12,14,14,14,14,14,14,14,24,24,24,24


In [136]:
568/100000*100

0.5680000000000001

In [137]:
df6 = df5.drop(labels=missing2.index,axis=1)

In [138]:
df7 = df6.fillna(df6.mean())

In [139]:
totalt = df7.isnull().sum().sort_values(ascending=False)
percent = (df7.isnull().sum()/df7.isnull().count()).sort_values(ascending=False)
missing_data4 = pd.concat([totalt, percent], axis=1, keys=['Total', 'Percent'])
missing_data4

,Total,Percent
Response,0,0.0
L3_S29_F3439,0,0.0
L3_S29_F3467,0,0.0
L3_S29_F3464,0,0.0
L3_S29_F3461,0,0.0
...,...,...
L3_S30_F3709,0,0.0
L3_S30_F3704,0,0.0
L3_S30_F3699,0,0.0
L3_S30_F3694,0,0.0


In [140]:
df0 = df7.loc[(df7['Response'] == 1)]
df0.shape

(568, 174)

In [141]:
df1 = df7.loc[(df7['Response'] == 0)]
df1.shape

(99432, 174)

In [142]:
df1=df1[0:568]
df1.shape

(568, 174)

In [143]:
frames = [df1, df0]
df8 = pd.concat(frames)
df8.shape

(1136, 174)

### Algoritmo equilibrando Response

In [144]:
from sklearn.preprocessing import StandardScaler
target = df8['Response'].values

select_features = missing_data4.index


scaler = StandardScaler()
dfScaled = scaler.fit_transform(df8[select_features])

In [145]:
from sklearn.model_selection import train_test_split
x_train1, x_test1, y_train1, y_test1 = train_test_split(dfScaled, target, test_size=0.2, random_state=42)

In [146]:
from sklearn.ensemble import RandomForestClassifier  # importing model to use for our prediction
from sklearn.metrics import classification_report

RFC1 = RandomForestClassifier(max_depth = 5, min_samples_split = 4, n_estimators = 500,random_state = 20, n_jobs = -1)
RFC1.fit(x_train1, y_train1) 
train_prediction1 = RFC1.predict(x_train1)
print("La accuracy es de {}.".format(accuracy_score(y_train1,train_prediction1)))
print(classification_report(y_train1, train_prediction1))

test_prediction1 = RFC1.predict(x_test1)
print("La accuracy es de {}.".format(accuracy_score(y_test1, test_prediction1)))
print(classification_report(y_test1, test_prediction1))

La accuracy es de 1.0.
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       444
           1       1.00      1.00      1.00       464

    accuracy                           1.00       908
   macro avg       1.00      1.00      1.00       908
weighted avg       1.00      1.00      1.00       908

La accuracy es de 0.9956140350877193.
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       124
           1       1.00      0.99      1.00       104

    accuracy                           1.00       228
   macro avg       1.00      1.00      1.00       228
weighted avg       1.00      1.00      1.00       228



En este caso teníamos un 0.56% de pieza mala. Por lo tanto buscamos una accuracy más alta que un 99.44%.

La accuracy en entrenamiento es del 100% por lo que podríamos concluir que hemos **sobreentrenado** el dataset. Pero en test tenemos una accuracy de **99.56%**, superior a lo que buscábamos.

 ## Discusión y Conclusiones

## Linias de desarrollo futuro

### Annexo 1


### Annexo 2